In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip3 install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=f0a1dc30c084d851cdb35caf4863cfaa2796eca7a7a131d9f0f917859d9ed49f
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession, Row
from pyspark.sql import types as T
from pyspark.sql import window as W
from pyspark.sql import functions as F

spark = SparkSession.builder \
        .master("local") \
        .appName("Colab") \
        .getOrCreate()

In [5]:
item = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/천재교육 프로젝트/Pyspark/item_his.csv', header=True)
study = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/천재교육 프로젝트/Pyspark/study_his.csv', header=True)
regdate = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/천재교육 프로젝트/Pyspark/regdate_his.csv', header=True)
member = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/천재교육 프로젝트/Pyspark/member.csv', header=True)
point = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/천재교육 프로젝트/Pyspark/point_his.csv', header=True)

- 1. member table에서 idx가 8000과 9000 사이인 row를 찾아주세요

- 2. member data에서 회원 상태가 몇명인지 내림차순으로 보여주세요.

- 3. 학습만료 회원들이 공부한 과목 리스트를 TOP3를 뽑아주세요

- 4. 등록일이 2023.03.15일인 유료회원들이 가장 많이 착용한 아이템 TOP3를 뽑아주세요

- 5. 레벨이 151~160 에 있는 유저들 중 딱 한 명씩만 등록한 날짜들을 구해주세요

- 6. item_his 테이블에서 레벨이 null인 유저가 가장 많은 날짜를 구해주세요

- 7. UDF를 활용하여 study의 pointnm 컬럼에 대해 공백을 모두 없애주세요

- 8. UDF를 활용하여 member의 grade 컬럼에 대해 '학년' 단어를 없애주세요

- 9. study_his 월별 point 발생 종류를 오름차순으로 순번매겨주세요

# 1. member table에서 idx가 8000과 9000 사이인 row를 찾아주세요

In [6]:
member.show()

+-----+---+--------+-------+
|  idx|sex|  status|  grade|
+-----+---+--------+-------+
| 6884| 여|유료회원|초3학년|
| 6331| 남|유료회원|초3학년|
|69294| 남|유료회원|초5학년|
|31531| 여|유료회원|초1학년|
|85784| 여|유료회원|초2학년|
|58058| 여|유료회원|초3학년|
|  777| 남|유료회원|초3학년|
| 5482| 여|유료회원|초2학년|
|63447| 남|유료회원|초2학년|
|54957| 남|유료회원|초4학년|
|55340| 남|유료회원|초2학년|
|72887| 여|유료회원|초3학년|
|40008| 여|유료회원|초3학년|
|57551| 남|유료회원|초5학년|
|58583| 여|유료회원|초3학년|
|32219| 남|유료회원|초1학년|
|13136| 여|유료회원|초4학년|
|22122| 남|유료회원|초6학년|
|18306| 남|유료회원|초4학년|
|28057| 남|유료회원|초3학년|
+-----+---+--------+-------+
only showing top 20 rows



In [7]:
member.printSchema()


root
 |-- idx: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- status: string (nullable = true)
 |-- grade: string (nullable = true)



In [8]:
member = member.withColumn('idx',  F.col('idx').cast(T.IntegerType()))


In [11]:
member.filter(F.col('idx').between(8000,9000)).orderBy(F.col('idx')).show()

+----+---+--------+-------+
| idx|sex|  status|  grade|
+----+---+--------+-------+
|8000| 남|유료회원|초2학년|
|8000| 남|유료회원|초3학년|
|8001| 여|유료회원|초5학년|
|8001| 여|유료회원|초4학년|
|8002| 여|유료회원|초3학년|
|8002| 여|유료회원|초4학년|
|8003| 여|유료회원|초3학년|
|8004| 남|유료회원|초3학년|
|8004| 남|유료회원|초2학년|
|8005| 여|유료회원|초3학년|
|8005| 여|유료회원|초2학년|
|8006| 여|유료회원|초4학년|
|8006| 여|유료회원|초3학년|
|8007| 남|유료회원|초1학년|
|8007| 남|유료회원|  0학년|
|8008| 남|유료회원|초2학년|
|8008| 남|유료회원|초3학년|
|8009| 남|유료회원|초6학년|
|8011| 남|유료회원|초1학년|
|8013| 남|유료회원|초4학년|
+----+---+--------+-------+
only showing top 20 rows



# 2. member data에서 회원 상태가 몇명인지 내림차순으로 보여주세요.

In [12]:
member.show()

+-----+---+--------+-------+
|  idx|sex|  status|  grade|
+-----+---+--------+-------+
| 6884| 여|유료회원|초3학년|
| 6331| 남|유료회원|초3학년|
|69294| 남|유료회원|초5학년|
|31531| 여|유료회원|초1학년|
|85784| 여|유료회원|초2학년|
|58058| 여|유료회원|초3학년|
|  777| 남|유료회원|초3학년|
| 5482| 여|유료회원|초2학년|
|63447| 남|유료회원|초2학년|
|54957| 남|유료회원|초4학년|
|55340| 남|유료회원|초2학년|
|72887| 여|유료회원|초3학년|
|40008| 여|유료회원|초3학년|
|57551| 남|유료회원|초5학년|
|58583| 여|유료회원|초3학년|
|32219| 남|유료회원|초1학년|
|13136| 여|유료회원|초4학년|
|22122| 남|유료회원|초6학년|
|18306| 남|유료회원|초4학년|
|28057| 남|유료회원|초3학년|
+-----+---+--------+-------+
only showing top 20 rows



In [ ]:
member_g = member.groupby('status').agg(F.count('status').alias('groupby count'))
member_g.orderBy('groupby count', ascending=False).show()

+--------+-------------+
|  status|groupby count|
+--------+-------------+
|유료회원|        97980|
|학습만료|          383|
|  재구매|          357|
|    신규|          261|
|    이월|           57|
|    복회|           17|
|이관유료|           15|
|    취소|           10|
+--------+-------------+



# 3. 학습만료 회원들이 공부한 과목 리스트를 TOP3를 뽑아주세요

In [ ]:
member.show()

+-----+---+--------+-------+
|  idx|sex|  status|  grade|
+-----+---+--------+-------+
| 6884| 여|유료회원|초3학년|
| 6331| 남|유료회원|초3학년|
|69294| 남|유료회원|초5학년|
|31531| 여|유료회원|초1학년|
|85784| 여|유료회원|초2학년|
|58058| 여|유료회원|초3학년|
|  777| 남|유료회원|초3학년|
| 5482| 여|유료회원|초2학년|
|63447| 남|유료회원|초2학년|
|54957| 남|유료회원|초4학년|
|55340| 남|유료회원|초2학년|
|72887| 여|유료회원|초3학년|
|40008| 여|유료회원|초3학년|
|57551| 남|유료회원|초5학년|
|58583| 여|유료회원|초3학년|
|32219| 남|유료회원|초1학년|
|13136| 여|유료회원|초4학년|
|22122| 남|유료회원|초6학년|
|18306| 남|유료회원|초4학년|
|28057| 남|유료회원|초3학년|
+-----+---+--------+-------+
only showing top 20 rows



In [ ]:
done_learning_df = member.filter(F.col('status') == '학습만료')
done_learning_df.show()

+-----+--------+--------+-------+
|  idx|     sex|  status|  grade|
+-----+--------+--------+-------+
|17790|      남|학습만료|초4학년|
|17410|      남|학습만료|초6학년|
|19738|      여|학습만료|초3학년|
| 4176|      남|학습만료|초6학년|
| 7587|      남|학습만료|초3학년|
|31427|      남|학습만료|초4학년|
|24024|      남|학습만료|초4학년|
|63924|      여|학습만료|초2학년|
|16438|      여|학습만료|초3학년|
|22061|정보없음|학습만료|초3학년|
|14457|      여|학습만료|초5학년|
| 7682|      여|학습만료|초2학년|
|22364|      남|학습만료|초4학년|
|61019|      남|학습만료|초3학년|
|18664|      여|학습만료|초3학년|
|34212|      남|학습만료|초2학년|
| 8469|      여|학습만료|초6학년|
|32506|      여|학습만료|초6학년|
|16570|      여|학습만료|초4학년|
|28025|      여|학습만료|초2학년|
+-----+--------+--------+-------+
only showing top 20 rows



In [ ]:
study.show()


+-----+-------+--------+-----------+
|  idx|proc_ym|proc_ymd|    pointnm|
+-----+-------+--------+-----------+
|88311| 202306|20230628|한글 스피치|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
|89542| 202305|20230509| 중학 3학년|
|23940| 202305|20230516| 중학 1학년|
|23940| 202305|20230516| 중학 1학년|
|40502| 202304|20230412| 중학 1학년|
| 1741| 202304|20230405| 중학 1학년|
|50161| 202304|20230428| 중학 1학년|
+-----+-------+--------+-----------+
only showing top 20 rows



In [ ]:
join_done = done_learning_df.join(study, (done_learning_df.idx == study.idx))

dup_df = join_done.dropDuplicates(subset=['idx'])
#해당 컬럼 122 중복제거 하는게 맞음?

In [ ]:
dup_df.count()

122

In [ ]:
filter_df = dup_df.groupby('pointnm').agg(F.count('pointnm').alias('count'))
filter_df.show()

+-----------------+-----+
|          pointnm|count|
+-----------------+-----+
|        일기 쓰기|    3|
|          AI 영어|    3|
|도전! AI 받아쓰기|   13|
|      중학 신입생|    3|
|          AI 국어|    2|
| 밀크T 지오그래픽|   12|
|       중학 1학년|    5|
|      한글 스피치|    2|
|       중학 2학년|    1|
|   AI 구구단 게임|   38|
|     AI서술형평가|   40|
+-----------------+-----+



In [ ]:
filter_df.orderBy('count', ascending=False).limit(3).show()

+-----------------+-----+
|          pointnm|count|
+-----------------+-----+
|     AI서술형평가|   40|
|   AI 구구단 게임|   38|
|도전! AI 받아쓰기|   13|
+-----------------+-----+



# 4. 등록일이 2023.03.15일인 유료회원들이 가장 많이 착용한 아이템 TOP3를 뽑아주세요

In [ ]:
regdate_filter = regdate.filter(F.col('regdate') == '20230315')
e_member = member.filter(F.col('status') == '유료회원')

filter = regdate_filter.join(e_member, regdate_filter.idx == e_member.idx).drop(e_member.idx)

filter.show()

+-----+--------+---+--------+-------+
|  idx| regdate|sex|  status|  grade|
+-----+--------+---+--------+-------+
|84668|20230315| 여|유료회원|초1학년|
|84755|20230315| 여|유료회원|초2학년|
|84746|20230315| 남|유료회원|초1학년|
|84860|20230315| 남|유료회원|초3학년|
|84669|20230315| 남|유료회원|  0학년|
|84730|20230315| 여|유료회원|초6학년|
|84872|20230315| 남|유료회원|초3학년|
|84689|20230315| 여|유료회원|초4학년|
|84818|20230315| 여|유료회원|초4학년|
|84727|20230315| 남|유료회원|초3학년|
|84858|20230315| 여|유료회원|초5학년|
|84678|20230315| 남|유료회원|초2학년|
|84819|20230315| 여|유료회원|초4학년|
|84815|20230315| 여|유료회원|초3학년|
|84707|20230315| 여|유료회원|초1학년|
|84878|20230315| 남|유료회원|초6학년|
|84705|20230315| 남|유료회원|초2학년|
|84664|20230315| 남|유료회원|초1학년|
|84686|20230315| 남|유료회원|초2학년|
|84859|20230315| 남|유료회원|  0학년|
+-----+--------+---+--------+-------+
only showing top 20 rows



In [ ]:
filter_item = filter.join(item, filter.idx==item.idx)

filter_item.show()

+-----+--------+---+--------+-------+-----+-----+-------+--------+----------+--------------+--------+-----+
|  idx| regdate|sex|  status|  grade|  idx|   lv|proc_ym|proc_ymd|  codename|   mascodename|payprice|price|
+-----+--------+---+--------+-------+-----+-----+-------+--------+----------+--------------+--------+-----+
|84746|20230315| 남|유료회원|초1학년|84746|143.0| 202305|20230526|      헤어|아바타파츠구분|     150|  150|
|84746|20230315| 남|유료회원|초1학년|84746|121.0| 202304|20230426|      신발|아바타파츠구분|      60|   60|
|84746|20230315| 남|유료회원|초1학년|84746|143.0| 202305|20230528|      얼굴|아바타파츠구분|     150|  150|
|84746|20230315| 남|유료회원|초1학년|84746|143.0| 202305|20230528|  액세서리|아바타파츠구분|      25|   25|
|84746|20230315| 남|유료회원|초1학년|84746| 97.0| 202304|20230416|상태메시지|  기타파츠구분|      50|   50|
|84746|20230315| 남|유료회원|초1학년|84746|141.0| 202305|20230522|      헤어|아바타파츠구분|     150|  150|
|84860|20230315| 남|유료회원|초3학년|84860|146.0| 202306|20230626|      얼굴|아바타파츠구분|     150|  150|
|84860|20230315| 남|유료회원|초3학년|84860| 88.0| 2

In [ ]:
counts_item = filter_item.groupby("codename").count()

counts_item.show()

+----------+-----+
|  codename|count|
+----------+-----+
|      신발|   44|
|      하의|   28|
|  액세서리|   35|
|      얼굴|   47|
|      헤어|   67|
|    코스튬|   24|
|상태메시지|   74|
|      상의|   29|
+----------+-----+



In [ ]:
top3_item = counts_item.orderBy(F.col('count').desc()).limit(3)
top3_item.show()

+----------+-----+
|  codename|count|
+----------+-----+
|상태메시지|   74|
|      헤어|   67|
|      얼굴|   47|
+----------+-----+



# 5. 레벨이 151~160 에 있는 유저들 중 딱 한 명씩만 등록한 날짜들을 구해주세요

In [13]:
item_his = item.filter(F.col('lv').between(151, 160))

DataFrame[idx: string, lv: string, proc_ym: string, proc_ymd: string, codename: string, mascodename: string, payprice: string, price: string]

In [15]:
item_reg = item_his.join(regdate, ['idx'])
item_reg = item_reg.drop_duplicates(subset=['idx', 'regdate'])
item_reg.show()

+-----+-----+-------+--------+----------+--------------+--------+-----+--------+
|  idx|   lv|proc_ym|proc_ymd|  codename|   mascodename|payprice|price| regdate|
+-----+-----+-------+--------+----------+--------------+--------+-----+--------+
|10009|159.0| 202304|20230414|      헤어|아바타파츠구분|     150|  150|20221206|
|10037|152.0| 202304|20230408|      신발|아바타파츠구분|      60|   60|20221206|
|10038|151.0| 202304|20230404|      상의|아바타파츠구분|     200|  200|20221206|
|10041|152.0| 202305|20230502|      신발|아바타파츠구분|      60|   60|20221206|
|10046|154.0| 202305|20230508|      신발|아바타파츠구분|      60|   60|20221206|
|10070|160.0| 202305|20230526|      상의|아바타파츠구분|     200|  200|20221206|
|10073|153.0| 202305|20230505|      신발|아바타파츠구분|      60|   60|20221206|
|10085|158.0| 202305|20230502|      헤어|아바타파츠구분|     150|  150|20221206|
|10089|153.0| 202304|20230405|      얼굴|아바타파츠구분|     150|  150|20221206|
| 1011|160.0| 202304|20230427|      하의|아바타파츠구분|     250|  250|20221206|
| 1013|158.0| 202304|20230412|상태메시지| 

In [17]:
item_reg.groupby("regdate").count().filter(F.col('count') == 1).orderBy(F.col('regdate')).show()

+--------+-----+
| regdate|count|
+--------+-----+
|20230129|    1|
|20230205|    1|
|20230211|    1|
|20230226|    1|
|20230305|    1|
|20230310|    1|
|20230311|    1|
|20230316|    1|
|20230320|    1|
|20230321|    1|
|20230325|    1|
|20230327|    1|
|20230330|    1|
|20230331|    1|
|20230410|    1|
|20230411|    1|
|20230420|    1|
|20230428|    1|
|20230505|    1|
+--------+-----+



In [ ]:
item_filter = item.filter(F.col('lv').between(151, 160)).orderBy(F.col('lv'))
item_filter.show()

+-----+-----+-------+--------+----------+--------------+--------+-----+
|  idx|   lv|proc_ym|proc_ymd|  codename|   mascodename|payprice|price|
+-----+-----+-------+--------+----------+--------------+--------+-----+
|10422|151.0| 202305|20230526|      헤어|아바타파츠구분|     150|  150|
|76267|151.0| 202304|20230418|      신발|아바타파츠구분|     150|  150|
|10422|151.0| 202305|20230526|  액세서리|아바타파츠구분|      50|   50|
|61823|151.0| 202304|20230418|  액세서리|아바타파츠구분|      25|   25|
|10422|151.0| 202305|20230526|      신발|아바타파츠구분|      60|   60|
|61823|151.0| 202304|20230418|      신발|아바타파츠구분|      60|   60|
|59678|151.0| 202305|20230530|      얼굴|아바타파츠구분|     150|  150|
|20036|151.0| 202304|20230403|      헤어|아바타파츠구분|     150|  150|
|59678|151.0| 202305|20230530|      신발|아바타파츠구분|      60|   60|
|10422|151.0| 202305|20230526|      얼굴|아바타파츠구분|     150|  150|
|59678|151.0| 202305|20230530|상태메시지|  기타파츠구분|      50|   50|
|10422|151.0| 202305|20230526|      신발|아바타파츠구분|      60|   60|
|10698|151.0| 202304|20230401|    

In [ ]:
regdate.count()

112568

In [ ]:
one_join = item_filter.join(regdate, item_filter.idx==regdate.idx).drop(regdate.idx)
one_join.show()

+-----+-----+-------+--------+----------+--------------+--------+-----+--------+
|  idx|   lv|proc_ym|proc_ymd|  codename|   mascodename|payprice|price| regdate|
+-----+-----+-------+--------+----------+--------------+--------+-----+--------+
|76038|159.0| 202306|20230613|      상의|아바타파츠구분|     200|  200|20230203|
|25146|159.0| 202305|20230515|      신발|아바타파츠구분|     150|  150|20221207|
|25146|159.0| 202305|20230515|상태메시지|  기타파츠구분|      50|   50|20221207|
|52108|159.0| 202305|20230515|      상의|아바타파츠구분|     200|  200|20221214|
|52108|159.0| 202305|20230515|    코스튬|아바타파츠구분|     300|  300|20221214|
|29907|158.0| 202304|20230404|      하의|아바타파츠구분|     250|  250|20221207|
|11987|159.0| 202304|20230404|      하의|아바타파츠구분|     250|  250|20221206|
|11987|159.0| 202304|20230404|      헤어|아바타파츠구분|     150|  150|20221206|
|11987|159.0| 202304|20230404|      신발|아바타파츠구분|      60|   60|20221206|
|11987|159.0| 202304|20230404|      신발|아바타파츠구분|      60|   60|20221206|
|11987|159.0| 202304|20230404|  액세서리|아바타

In [ ]:
one_countss = one_join.groupby("regdate").count()
one_countss.show()

+--------+-----+
| regdate|count|
+--------+-----+
|20230127|   33|
|20230203|   39|
|20230101|   36|
|20230105|   56|
|20230201|   26|
|20230110|   27|
|20230220|    9|
|20230128|   35|
|20230120|   31|
|20230115|   20|
|20230116|   64|
|20230327|    3|
|20230108|   15|
|20221206| 5397|
|20230304|   15|
|20221224|   52|
|20230222|   10|
|20221219|  204|
|20230113|   51|
|20230301|   15|
+--------+-----+
only showing top 20 rows



In [ ]:
one_countss = one_countss.filter(F.col('count') == 1)

one_countss.show()

+--------+-----+
| regdate|count|
+--------+-----+
|20230321|    1|
|20230129|    1|
|20230325|    1|
|20230226|    1|
|20230310|    1|
|20230410|    1|
|20230311|    1|
|20230331|    1|
+--------+-----+



# 6. item_his 테이블에서 레벨이 null인 유저가 가장 많은 날짜를 구해주세요

In [ ]:
item_f = item.filter(F.col('lv').isNull())
item_f.show()

+-----+----+-------+--------+--------+--------------+--------+-----+
|  idx|  lv|proc_ym|proc_ymd|codename|   mascodename|payprice|price|
+-----+----+-------+--------+--------+--------------+--------+-----+
|92027|NULL| 202305|20230505|  코스튬|아바타파츠구분|     300|  300|
|92131|NULL| 202305|20230507|  코스튬|아바타파츠구분|     300|  300|
|91962|NULL| 202305|20230506|  코스튬|아바타파츠구분|     300|  300|
|94835|NULL| 202306|20230605|  코스튬|아바타파츠구분|     300|  300|
|94835|NULL| 202306|20230605|  코스튬|아바타파츠구분|     300|  300|
|91129|NULL| 202304|20230427|  코스튬|아바타파츠구분|     300|  300|
|91129|NULL| 202304|20230427|  코스튬|아바타파츠구분|     300|  300|
|91129|NULL| 202304|20230427|  코스튬|아바타파츠구분|     300|  300|
|93173|NULL| 202305|20230517|  코스튬|아바타파츠구분|     300|  300|
|96008|NULL| 202306|20230618|  코스튬|아바타파츠구분|     300|  300|
|96008|NULL| 202306|20230618|  코스튬|아바타파츠구분|     300|  300|
|96008|NULL| 202306|20230618|  코스튬|아바타파츠구분|     300|  300|
|91835|NULL| 202305|20230503|  코스튬|아바타파츠구분|     300|  300|
|91835|NULL| 202305|202305

In [ ]:
item_f_d = item_f.groupBy('proc_ymd').count()
item_f_d.show()

+--------+-----+
|proc_ymd|count|
+--------+-----+
|20230508|    9|
|20230616|    2|
|20230408|    5|
|20230414|    4|
|20230405|    7|
|20230603|    2|
|20230617|    1|
|20230502|    8|
|20230522|    4|
|20230612|    1|
|20230511|    3|
|20230505|   11|
|20230427|    6|
|20230402|    5|
|20230501|    3|
|20230513|    3|
|20230406|    1|
|20230428|    4|
|20230419|    2|
|20230519|    7|
+--------+-----+
only showing top 20 rows



In [ ]:
# top3_item = counts_item.orderBy(F.col('count').desc()).limit(3)
item_f_d.orderBy(F.col('count').desc()).limit(1).show()

+--------+-----+
|proc_ymd|count|
+--------+-----+
|20230504|   16|
+--------+-----+



# 7. UDF를 활용하여 study의 pointnm 컬럼에 대해 공백을 모두 없애주세요

In [ ]:
study.show()

+-----+-------+--------+-----------+
|  idx|proc_ym|proc_ymd|    pointnm|
+-----+-------+--------+-----------+
|88311| 202306|20230628|한글 스피치|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
|89542| 202305|20230509| 중학 3학년|
|23940| 202305|20230516| 중학 1학년|
|23940| 202305|20230516| 중학 1학년|
|40502| 202304|20230412| 중학 1학년|
| 1741| 202304|20230405| 중학 1학년|
|50161| 202304|20230428| 중학 1학년|
+-----+-------+--------+-----------+
only showing top 20 rows



In [ ]:
def remove_whitespace(text):
    return text.replace(" ", "")

# StringType으로 반환하는 UDF를 등록합니다.

remove_whitespace_udf = F.udf(remove_whitespace, returnType=T.StringType())


# DataFrame에 UDF를 적용하여 새로운 컬럼을 추가합니다.
study_filter = study.withColumn('pointnm' ,remove_whitespace_udf("pointnm"))

# 결과를 출력합니다.
study_filter.show()



+-----+-------+--------+----------+
|  idx|proc_ym|proc_ymd|   pointnm|
+-----+-------+--------+----------+
|88311| 202306|20230628|한글스피치|
| 8604| 202306|20230606| 중학3학년|
| 8604| 202306|20230606| 중학3학년|
| 8604| 202306|20230606| 중학3학년|
| 8604| 202306|20230606| 중학3학년|
| 8604| 202306|20230606| 중학3학년|
| 8604| 202306|20230606| 중학3학년|
| 8604| 202306|20230606| 중학3학년|
| 8604| 202306|20230606| 중학3학년|
| 8604| 202306|20230606| 중학3학년|
| 8604| 202306|20230606| 중학3학년|
| 8604| 202306|20230606| 중학3학년|
| 8604| 202306|20230606| 중학3학년|
| 8604| 202306|20230606| 중학3학년|
|89542| 202305|20230509| 중학3학년|
|23940| 202305|20230516| 중학1학년|
|23940| 202305|20230516| 중학1학년|
|40502| 202304|20230412| 중학1학년|
| 1741| 202304|20230405| 중학1학년|
|50161| 202304|20230428| 중학1학년|
+-----+-------+--------+----------+
only showing top 20 rows



# 8. UDF를 활용하여 member의 grade 컬럼에 대해 '학년' 단어를 없애주세요

In [ ]:
member.show()

+-----+---+--------+-------+
|  idx|sex|  status|  grade|
+-----+---+--------+-------+
| 6884| 여|유료회원|초3학년|
| 6331| 남|유료회원|초3학년|
|69294| 남|유료회원|초5학년|
|31531| 여|유료회원|초1학년|
|85784| 여|유료회원|초2학년|
|58058| 여|유료회원|초3학년|
|  777| 남|유료회원|초3학년|
| 5482| 여|유료회원|초2학년|
|63447| 남|유료회원|초2학년|
|54957| 남|유료회원|초4학년|
|55340| 남|유료회원|초2학년|
|72887| 여|유료회원|초3학년|
|40008| 여|유료회원|초3학년|
|57551| 남|유료회원|초5학년|
|58583| 여|유료회원|초3학년|
|32219| 남|유료회원|초1학년|
|13136| 여|유료회원|초4학년|
|22122| 남|유료회원|초6학년|
|18306| 남|유료회원|초4학년|
|28057| 남|유료회원|초3학년|
+-----+---+--------+-------+
only showing top 20 rows



In [ ]:
def remove_grade_word(text):
    return text.replace("학년", "")

# StringType으로 반환하는 UDF를 등록합니다.
remove_grade_word_udf = F.udf(remove_grade_word, returnType=T.StringType())

# DataFrame에 UDF를 적용하여 새로운 컬럼을 추가합니다.
member_filter = member.withColumn("grade", remove_grade_word_udf("grade"))

# 결과를 출력합니다.
member_filter.show()

+-----+---+--------+-----+
|  idx|sex|  status|grade|
+-----+---+--------+-----+
| 6884| 여|유료회원|  초3|
| 6331| 남|유료회원|  초3|
|69294| 남|유료회원|  초5|
|31531| 여|유료회원|  초1|
|85784| 여|유료회원|  초2|
|58058| 여|유료회원|  초3|
|  777| 남|유료회원|  초3|
| 5482| 여|유료회원|  초2|
|63447| 남|유료회원|  초2|
|54957| 남|유료회원|  초4|
|55340| 남|유료회원|  초2|
|72887| 여|유료회원|  초3|
|40008| 여|유료회원|  초3|
|57551| 남|유료회원|  초5|
|58583| 여|유료회원|  초3|
|32219| 남|유료회원|  초1|
|13136| 여|유료회원|  초4|
|22122| 남|유료회원|  초6|
|18306| 남|유료회원|  초4|
|28057| 남|유료회원|  초3|
+-----+---+--------+-----+
only showing top 20 rows



# 9. study_his 월별 point 발생 종류를 오름차순으로 순번매겨주세요

In [ ]:
study.show()

+-----+-------+--------+-----------+
|  _c0|    _c1|     _c2|      study|
+-----+-------+--------+-----------+
|  idx|proc_ym|proc_ymd|    pointnm|
|88311| 202306|20230628|한글 스피치|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
|89542| 202305|20230509| 중학 3학년|
|23940| 202305|20230516| 중학 1학년|
|23940| 202305|20230516| 중학 1학년|
|40502| 202304|20230412| 중학 1학년|
| 1741| 202304|20230405| 중학 1학년|
+-----+-------+--------+-----------+
only showing top 20 rows



In [ ]:
from pyspark.sql import window as W
study.show()

+-----+-------+--------+-----------+
|  idx|proc_ym|proc_ymd|    pointnm|
+-----+-------+--------+-----------+
|88311| 202306|20230628|한글 스피치|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
|89542| 202305|20230509| 중학 3학년|
|23940| 202305|20230516| 중학 1학년|
|23940| 202305|20230516| 중학 1학년|
|40502| 202304|20230412| 중학 1학년|
| 1741| 202304|20230405| 중학 1학년|
|50161| 202304|20230428| 중학 1학년|
+-----+-------+--------+-----------+
only showing top 20 rows



In [18]:
study.show()

+-----+-------+--------+-----------+
|  idx|proc_ym|proc_ymd|    pointnm|
+-----+-------+--------+-----------+
|88311| 202306|20230628|한글 스피치|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
| 8604| 202306|20230606| 중학 3학년|
|89542| 202305|20230509| 중학 3학년|
|23940| 202305|20230516| 중학 1학년|
|23940| 202305|20230516| 중학 1학년|
|40502| 202304|20230412| 중학 1학년|
| 1741| 202304|20230405| 중학 1학년|
|50161| 202304|20230428| 중학 1학년|
+-----+-------+--------+-----------+
only showing top 20 rows



In [22]:
from pyspark.sql import window as W

study_windoww = study.drop_duplicates(subset=['proc_ym', 'pointnm'])

In [23]:
study_window_var1 = W.Window.partitionBy('proc_ym').orderBy('pointnm')

In [25]:
study_windoww.withColumn('rank', F.row_number().over(study_window_var1)).show()

+-----+-------+--------+-----------------+----+
|  idx|proc_ym|proc_ymd|          pointnm|rank|
+-----+-------+--------+-----------------+----+
|84670| 202304|20230422|   AI 구구단 게임|   1|
|34403| 202304|20230409|          AI 국어|   2|
|87085| 202304|20230410|          AI 영어|   3|
|59479| 202304|20230404|     AI서술형평가|   4|
|78295| 202304|20230408|도전! AI 받아쓰기|   5|
|22417| 202304|20230416| 밀크T 지오그래픽|   6|
|14973| 202304|20230417|         받아쓰기|   7|
|73525| 202304|20230411|        일기 쓰기|   8|
|42440| 202304|20230420|     자기소개하기|   9|
|40502| 202304|20230412|       중학 1학년|  10|
|10338| 202304|20230416|       중학 2학년|  11|
|13597| 202304|20230414|       중학 3학년|  12|
|12756| 202304|20230426|      중학 신입생|  13|
| 8947| 202304|20230422|        중학 특강|  14|
|87623| 202304|20230415| 학교 공부 맛보기|  15|
|74751| 202304|20230429|        학교 체험|  16|
|42879| 202304|20230421|      한글 스피치|  17|
|78110| 202305|20230519|   AI 구구단 게임|   1|
|64348| 202305|20230509|          AI 국어|   2|
|85885| 202305|20230530|     